In [1]:
!pip install webdriver-manager

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\moren\AppData\Local\Temp\ipykernel_37100\2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Player Value

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the Chrome driver using WebDriver Manager
driver = webdriver.Chrome(ChromeDriverManager().install())

# Navigate to the initial webpage where the list of products is
# Replace this URL with the actual URL of your product list page
driver.get('https://www.transfermarkt.ch/servette-fc/kader/verein/61/plus/0/galerie/0?saison_id=2023')
time.sleep(2)  # Wait for page to load

# Wait for the iframe to be present and switch to it
wait = WebDriverWait(driver, 10)
iframe = wait.until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953386")))
driver.switch_to.frame(iframe)

# Now wait for the 'Akzeptieren' button to be clickable inside the iframe
accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'accept')]")))
accept_button.click()

# Switch back to the main document
driver.switch_to.default_content()

# Find the table by its XPath or CSS Selector
table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')

# Initialize a list to store all rows of the table
table_data = []

# Locate all rows of the table
rows = table.find_elements(By.TAG_NAME, "tr")

# Loop through each row in the table
for row in rows:
    # Get the text of each cell in the row
    row_data = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
    table_data.append(row_data)

# Convert the list of rows to a pandas DataFrame
df = pd.DataFrame(table_data)

# Drop the first three columns and the sixth and seventh columns (adjust indices as needed)
# Columns are 0-indexed, so column 1 is at index 0, column 6 is at index 5, etc.
df.drop(df.columns[[0, 1, 2, 5, 6]], axis=1, inplace=True)

# Rename the remaining columns
df.columns = ['Name', 'Position', 'Contract', 'Value']

# Drop rows where all the elements are 'None'
df.dropna(how='all', inplace=True)

# Optionally, save the DataFrame to a CSV file
df.to_csv('path_to_save.csv', index=False)

# Close the driver after scraping is done
driver.quit()

# Print a success message
print("Webscraping successfully completed")
df

C:\Users\moren\AppData\Local\Temp\ipykernel_37100\2811746981.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Webscraping successfully completed


,Name,Position,Contract,Value
1,Joël Mall,Torwart,30.06.2025,500 Tsd. €
4,Jérémy Frick,Torwart,30.06.2027,500 Tsd. €
7,Léo Besson,Torwart,30.06.2026,100 Tsd. €
10,Nicolas Vouilloz,Innenverteidiger,30.06.2026,"1,70 Mio. €"
13,Yoan Severin,Innenverteidiger,30.06.2026,"1,10 Mio. €"
16,Jérôme Onguéné,Innenverteidiger,30.06.2024,"1,00 Mio. €"
19,Anthony Baron,Innenverteidiger,30.06.2024,300 Tsd. €
22,Steve Rouiller,Innenverteidiger,30.06.2025,200 Tsd. €
25,Baba Souare,Innenverteidiger,30.06.2024,200 Tsd. €
28,Noah Henchoz,Innenverteidiger,30.06.2025,100 Tsd. €


In [53]:
import sys
sys.version

'3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]'

# League Table

In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the Chrome driver using WebDriver Manager
driver = webdriver.Chrome(ChromeDriverManager().install())

# Function to extract the entire table for a given season
def extract_table_data(season, first_time=True):
    # Construct URL for the given season
    url = f'https://www.transfermarkt.ch/super-league/tabelle/wettbewerb/C1/saison_id/{season}'
    
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Wait for page to load

    if first_time:
        # Wait for the iframe to be present and switch to it
        wait = WebDriverWait(driver, 10)
        try:
            iframe = wait.until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953386")))
            driver.switch_to.frame(iframe)

            # Now wait for the 'Akzeptieren' button to be clickable inside the iframe
            accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'accept')]")))
            accept_button.click()

            # Switch back to the main document
            driver.switch_to.default_content()
        except TimeoutException:
            # If the iframe does not appear, just continue
            pass

    # Find the table by its XPath
    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')

    # Initialize a list to store all rows of the table
    table_data = []

    # Locate all rows of the table and extract the text
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        row_data = []
        # Initialize variables for symbols
        meister_symbol = ''
        cupsieger_symbol = ''

        cells = row.find_elements(By.TAG_NAME, "td")
        for cell in cells:
            text = cell.text.strip()
            
            # Check for symbols within the cell
            meister_img = cell.find_elements(By.XPATH, ".//a[2]/img")
            if meister_img:
                meister_symbol = meister_img[0].get_attribute('alt')
            cupsieger_img = cell.find_elements(By.XPATH, ".//a[3]/img")
            if cupsieger_img:
                cupsieger_symbol = cupsieger_img[0].get_attribute('alt')
            
            # Add text if it's not an empty string after removing symbols
            if text != '':
                row_data.append(text)
        
        # Ensure we have the right number of data columns before adding symbols
        while len(row_data) < 10:
            row_data.append('')
        
        # Add symbols
        row_data.extend([meister_symbol, cupsieger_symbol, season])
        table_data.append(row_data)

    return table_data[1:]  # Exclude the header row

# Initialize a list to store data for all seasons
all_seasons_data = []

# Loop through the desired seasons
first_time = True
for i in range(2023, 2013, -1):  # Adjust as needed
    season_data = extract_table_data(i, first_time)
    all_seasons_data.extend(season_data)
    first_time = False

# Convert the list of season data to a pandas DataFrame
df = pd.DataFrame(all_seasons_data, columns=['Rank', 'Club', 'Played', 'Wins', 'Draws', 'Losses', 'Goals', 'Goal Difference', 'Points', 'Symbol 1', 'Title 1', 'Title 2', 'Season'])

# Drop the 'Symbol 1' column from the DataFrame
df = df.drop('Symbol 1', axis=1)

# Close the driver after scraping is done
driver.quit()

# Optionally, save the DataFrame to a CSV file
df.to_csv('full_league_data.csv', index=False)

# Print a success message
print("Webscraping successfully completed")
print(df)


C:\Users\moren\AppData\Local\Temp\ipykernel_37100\3657351159.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Webscraping successfully completed
    Rank            Club Played Wins Draws Losses  Goals Goal Difference  \
0      1  BSC Young Boys     18   11     5      2  40:17              23   
1      2   FC St. Gallen     18   10     3      5  33:21              12   
2      3       FC Zürich     18    8     7      3  31:17              14   
3      4     Servette FC     18    8     7      3  30:22               8   
4      5       FC Lugano     18    8     2      8  32:30               2   
..   ...             ...    ...  ...   ...    ...    ...             ...   
97     6   FC St. Gallen     36   13     8     15  57:65              -8   
98     7         FC Sion     36   12     9     15  47:48              -1   
99     8    Grasshoppers     36   11    10     15  50:56              -6   
100    9        FC Vaduz     36    7    10     19  28:59             -31   
101   10        FC Aarau     36    6    12     18  31:64             -33   

    Points                         Title 1          

In [5]:
# Path to CSV-File
file_path = 'full_league_data.csv'  # Ersetzen Sie dies durch den tatsächlichen Dateipfad

# Read DataFrames
df = pd.read_csv(file_path)

# Replace 'Ã¼' with 'ü' in the 'Club' column
df['Club'] = df['Club'].str.replace('Ã¼', 'ü')


# Split the 'Goals' column into 'Goals Scored' and 'Goals Conceded'
goals_split = df['Goals'].str.split(':', expand=True)
df['Goals Scored'] = goals_split[0]
df['Goals Conceded'] = goals_split[1]

# It seems the 'Goals Conceded' may contain ':00' based on the screenshot you provided. Let's remove it.
df['Goals Conceded'] = df['Goals Conceded'].str.replace(':00', '', regex=False)

# Remove the original 'Goals' column
df.drop('Goals', axis=1, inplace=True)

# Reorder the columns to reflect the new structure
columns_ordered = ['Rank', 'Club', 'Played', 'Wins', 'Draws', 'Losses', 'Goals Scored', 'Goals Conceded', 'Goal Difference', 'Points', 'Title 1', 'Title 2', 'Season']
df = df[columns_ordered]

# Saving CSV-File
df.to_csv('full_league_data_cleaned.csv', index=False)


# Team Value

In [13]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the Chrome driver using WebDriver Manager
driver = webdriver.Chrome(ChromeDriverManager().install())

# Function to extract the entire table for a given season
def extract_table_data(season, first_time=True):
    # Construct URL for the given season
    url = f'https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/saison_id/{season}'
    
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Wait for page to load

    if first_time:
        # Wait for the iframe to be present and switch to it
        wait = WebDriverWait(driver, 10)
        try:
            iframe = wait.until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953386")))
            driver.switch_to.frame(iframe)

            # Now wait for the 'Akzeptieren' button to be clickable inside the iframe
            accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'accept')]")))
            accept_button.click()

            # Switch back to the main document
            driver.switch_to.default_content()
        except TimeoutException:
            # If the iframe does not appear, just continue
            pass

    # Find the table by its XPath
    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')

    # Initialize a list to store all rows of the table
    table_data = []

    # Locate all rows of the table and extract the text
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        row_data = []
        # Initialize variables for symbols
        meister_symbol = ''
        cupsieger_symbol = ''

        cells = row.find_elements(By.TAG_NAME, "td")
        for cell in cells:
            text = cell.text.strip()
            
            # Check for symbols within the cell
            meister_img = cell.find_elements(By.XPATH, ".//a[2]/img")
            if meister_img:
                meister_symbol = meister_img[0].get_attribute('alt')
            cupsieger_img = cell.find_elements(By.XPATH, ".//a[3]/img")
            if cupsieger_img:
                cupsieger_symbol = cupsieger_img[0].get_attribute('alt')
            
            # Add text if it's not an empty string after removing symbols
            if text != '':
                row_data.append(text)
        
        # Ensure we have the right number of data columns before adding symbols
        while len(row_data) < 10:
            row_data.append('')
        
        # Add symbols
        row_data.extend([meister_symbol, cupsieger_symbol, season])
        table_data.append(row_data)

    return table_data[1:]  # Exclude the header row

# Initialize a list to store data for all seasons
all_seasons_data = []

# Loop through the desired seasons
first_time = True
for i in range(2023, 2013, -1):  # Adjust as needed
    season_data = extract_table_data(i, first_time)
    all_seasons_data.extend(season_data)
    first_time = False

# Convert the list of season data to a pandas DataFrame
df = pd.DataFrame(all_seasons_data, columns=['Club', 'Squad', 'Age', 'Legionaere', 'Value', 'Total Value', 'Test', 'Test 1', 'Test 2', 'Test 3', 'Title 1', 'Title 2', 'Season'])

if 'Symbol 1' in df.columns:
    df = df.drop('Symbol 1', axis=1)
    
# Drop the 'Symbol 1' column from the DataFrame
df = df.drop('Test', axis=1)
df = df.drop('Test 1', axis=1)
df = df.drop('Test 2', axis=1)
df = df.drop('Test 3', axis=1)

# Close the driver after scraping is done
driver.quit()

# Optionally, save the DataFrame to a CSV file
df.to_csv('full_value_data.csv', index=False)

# Print a success message
print("Webscraping successfully completed")
print(df)


C:\Users\moren\AppData\Local\Temp\ipykernel_37100\2350013595.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Webscraping successfully completed
                   Club       Squad   Age  Legionaere          Value  \
0                   362  25,0 Jahre   204  857 Tsd. €  310,23 Mio. €   
1        BSC Young Boys          26  25,0          12    2,43 Mio. €   
2             FC Lugano          35  23,9          22     962 Tsd. €   
3         FC Basel 1893          30  23,4          20    1,05 Mio. €   
4             FC Luzern          29  24,0          13    1,01 Mio. €   
..                  ...         ...   ...         ...            ...   
107           FC Luzern          32  25,4          15     589 Tsd. €   
108  FC St. Gallen 1879          30  25,9          15     518 Tsd. €   
109             FC Thun          31  25,2           8     458 Tsd. €   
110            FC Aarau          38  27,5          21     295 Tsd. €   
111            FC Vaduz          27  26,1          16     374 Tsd. €   

      Total Value                          Title 1                    Title 2  \
0                  

# Market Value Over Time

In [22]:
import time
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# Function to extract the entire table for a given date
def extract_table_data(date, first_time):
    # Construct URL for the given date
    url = f'https://www.transfermarkt.ch/super-league/marktwerteverein/wettbewerb/C1/plus/?stichtag={date.strftime("%Y-%m-%d")}'
    
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Wait for the page to load
    
    if first_time:
        # Wait for the iframe to be present and switch to it
        wait = WebDriverWait(driver, 10)
        try:
            iframe = wait.until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953386")))
            driver.switch_to.frame(iframe)

            # Now wait for the 'Akzeptieren' button to be clickable inside the iframe
            accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'accept')]")))
            accept_button.click()

            # Switch back to the main document
            driver.switch_to.default_content()
        except TimeoutException:
            # If the iframe does not appear, just continue
            pass

    # Find the table by its XPath - Update this if the table structure is different
    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
    
    # Initialize a list to store all rows of the table
    table_data = []
    
    # Locate all rows of the table and extract the text
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        # Skip the header row
        if row.get_attribute('class') == 'thead':
            continue
        row_data = []
        cells = row.find_elements(By.TAG_NAME, "td")
        for cell in cells:
            text = cell.text.strip()
            row_data.append(text)
        
        # Add date to each row
        row_data.append(date.strftime("%Y-%m-%d"))
        table_data.append(row_data)

    return table_data

# Initialize the Chrome driver using WebDriver Manager
driver = webdriver.Chrome(ChromeDriverManager().install())

# Initialize a list to store data for all dates
all_dates_data = []

# Set the starting date to the 1st of the current month and then adjust to the 15th of the previous month
start_date = datetime.strptime("2024-01-01", "%Y-%m-%d")
start_date = (start_date.replace(day=1) - timedelta(days=1)).replace(day=15)

# Loop to alternate between the 15th and 1st of each month
first_time = True
for _ in range(24):  # Two iterations for each month (15th and 1st), hence 24 for 12 months
    # Extract data for the current date
    date_data = extract_table_data(start_date, first_time)
    all_dates_data.extend(date_data)
    first_time = False

    # Alternate between the 15th and the 1st of the month
    if start_date.day == 15:
        start_date = start_date.replace(day=1)
    else:
        # Move to the 15th of the previous month
        start_date = (start_date - timedelta(days=1)).replace(day=15)

# Convert the list of date data to a pandas DataFrame
# Adjust the column names based on the structure of the table you are scraping
df = pd.DataFrame(all_dates_data, columns=['Rank', 'Column2', 'Club', 'League', 'Value', 'Current Value', 'Change', 'As of'])

# Drop the 'Symbol 1' column from the DataFrame
df = df.drop('Column2', axis=1)

# Close the driver after scraping is done
driver.quit()

# Save the DataFrame to a CSV file
df.to_csv('full_value_dates.csv', index=False)

# Print a success message
print("Webscraping successfully completed")
print(df)


C:\Users\moren\AppData\Local\Temp\ipykernel_37100\3556194530.py:62: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Webscraping successfully completed
           Rank                       Club            League          Value  \
0    2023-12-15                       None              None           None   
1                Gesamtwert aller Vereine:                    298,30 Mio. €   
2             1             BSC Young Boys      Super League   58,30 Mio. €   
3             2                  FC Lugano      Super League   29,50 Mio. €   
4             3              FC Basel 1893      Super League   33,80 Mio. €   
..          ...                        ...               ...            ...   
331           8                  FC Zürich      Super League   25,93 Mio. €   
332           9    Grasshopper Club Zürich      Super League   21,80 Mio. €   
333          10    FC Stade-Lausanne-Ouchy  Challenge League    5,48 Mio. €   
334          11           Yverdon Sport FC  Challenge League    4,90 Mio. €   
335          12              FC Winterthur      Super League    9,53 Mio. €   

     Current Val